In [ ]:
# from Modules.LSH import semantic_query_lsh
# from Modules.LSH import LSH


# import numpy as np
# import os

In [ ]:
# file_path = "./random_data.txt"
# read_data = np.loadtxt(file_path)
# plane_norms = LSH(read_data, 8)
# query=[read_data[0]]
# folder_name = "bucket_files"
# result = semantic_query_lsh(query, plane_norms,folder_name)

In [23]:
%load_ext autoreload
%autoreload 2


from utils import *
from Modules.LSH import*
from api import *
from evaluation import *
from worst_case_implementation import VecDBWorst


# datafile_path="../DataBase/random_data_10000.txt"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
data_file='./DataBase/data.bin'
Level_1_path='./DataBase/Level1'
Level_2_path='./DataBase/Level2'
Level_3_path='./DataBase/Level3'

Level_1_nbits=8
Level_2_nbits=3
Level_3_nbits=3

data_api = DataApi(data_file)
data_api.generate_data_file(10000)

Inserted 10000 records. Total records inserted: 10000
Insertion complete.


In [33]:

# Test LSH_index
# Read Data From File
read_data = data_api.get_top_k_records(10000)


# Layer(1)
level_1_in=read_data
# TODO: Save Planes to be used in query Search
level_1_planes=LSH_index(data=level_1_in, nbits=Level_1_nbits,index_path=Level_1_path)

In [34]:
# Layer(2)
# On Each Bucket Apply LSH

# List all files in the directory
files = os.listdir(Level_1_path)

# TODO: Save Planes to be used in query Search
level_2_planes={}

# Loop over the files
for file_name in files:
    file_path = os.path.join(Level_1_path, file_name)
    
    if os.path.isfile(file_path):
        # Read Data
        read_data_2 = np.loadtxt(file_path,dtype=int,ndmin=1)

        level_2_in=data_api.get_multiple_records_by_ids(read_data_2-1)
        # level_2_in = array_to_dictionary(values=vectors,keys=np.hstack(read_data_2))

        # # Apply LSH on this Bucket
        # level_2=arr[level_1]
        level_2_planes[file_name[:-4]]=LSH_index(data=level_2_in.values(), nbits=Level_2_nbits,index_path=Level_2_path+'/' + file_name[:-4])



In [9]:
# Layer(3)
# On Each Bucket Apply LSH

# List all files in the directory
folders = os.listdir(Level_2_path)

# TODO: Save Planes to be used in query Search
level_3_planes={}
# file_3=folder{}
# Loop over the folders
for folder_name in folders:
    level_3_planes[folder_name]={}
    folder_path = os.path.join(Level_2_path, folder_name)
    files = os.listdir(folder_path)
    # Loop over the files
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path):
            # Read Data
            read_data_3 = np.loadtxt(file_path,dtype=int,ndmin=1)

            level_3_in=data_api.get_multiple_records_by_ids(read_data_3)

            # # Apply LSH on this Bucket
            level_3_planes[folder_name][file_name[:-4]]=LSH_index(data=level_3_in.values(), nbits=Level_3_nbits,index_path=Level_3_path+'/'+folder_name+'/' + file_name[:-4])


In [22]:
# Query
query=data_api.get_record_by_id(5)[5]['embed']
# Layer (1)
bucket_1,index_result_1 = semantic_query_lsh(query=query,plane_norms=level_1_planes,index_path=Level_1_path)
print("bucket of level 1: ",bucket_1)
# print("Length of level 1",len(index_result_1))
# print("Indices of level 1",index_result_1)
print("=====================================")

# Layer(2)
bucket_2,index_result_2 = semantic_query_lsh(query=query,plane_norms=level_2_planes[bucket_1],index_path=Level_2_path+"/"+bucket_1)
print("bucket of level 2: ",bucket_2)
# print("Length of level 2",len(index_result_2))
# print("Indices of level 2",index_result_2)
print("=====================================")

# Layer(3)
bucket_3,index_result_3 = semantic_query_lsh(query=query,plane_norms=level_3_planes[bucket_1][bucket_2],index_path=Level_3_path+"/"+bucket_1+'/'+bucket_2)
print("bucket of level 3: ",bucket_3)
print("Length of level 3",len(index_result_3))
print("Indices of level 3",index_result_3)


print("=====================================")

# get top 10 results from the last layer
index_result_3=data_api.get_multiple_records_by_ids(index_result_3)
level3_res_vectors=[entry['embed'] for entry in index_result_3.values()]
top_result,_=get_top_k_similar(query,index_result_3,10)
# print("Top k results: ",top_result[0])
# print("=====================================")
# # get the intersection of the two lists level 2 and level3
# count = sum(element in index_result_2 for element in index_result_3)
# print("Intersection of the two layers: ",count)



bucket of level 1:  11110011
bucket of level 2:  001
bucket of level 3:  100
Length of level 3 189
Indices of level 3 [  36   51  104  159  266  357  372  385  434  465  510  671  702  707
  720  822  824  834  863  938 1034 1044 1165 1248 1264 1438 1505 1565
 1613 1683 1712 1719 1771 1798 1812 1843 1953 2191 2238 2266 2330 2353
 2594 2602 2624 2669 2730 2744 2825 2880 2894 2915 2942 2944 3080 3168
 3286 3351 3490 3645 3648 3735 3798 3851 3859 3911 3986 4026 4030 4065
 4121 4134 4187 4211 4232 4260 4391 4399 4476 4477 4489 4492 4545 4554
 4591 4605 4660 4792 4905 4937 4953 4954 4970 4986 4987 5228 5249 5329
 5398 5454 5471 5495 5584 5708 5712 5725 5744 5799 5899 5900 5908 5952
 5987 6049 6072 6096 6144 6184 6209 6287 6344 6399 6479 6495 6536 6544
 6662 6693 6848 6880 6915 6962 7080 7085 7187 7199 7213 7240 7390 7404
 7417 7442 7531 7538 7554 7584 7625 7664 7708 7721 7765 7768 7808 7827
 7955 8101 8170 8279 8284 8380 8444 8446 8454 8481 8552 8560 8565 8586
 8676 8700 8761 8792 8912 8935

TypeError: unhashable type: 'slice'

In [ ]:
np_rows = np.array([record['embed'] for record in read_data if 'embed' in record])
# temp=[5, 966, 536, 1088, 5073, 5549]
index_result_3_minus_one = [id - 1 for id in top_results[0]]
res=run_queries(index_result_3_minus_one, np_rows, top_k=len(top_results), num_runs=1,query=np.array([query]))
print(eval(res))

In [ ]:
db = VecDBWorst()
# records_np = np.random.random((10000, 70))
records_np = np.array([record['embed'] for record in read_data if 'embed' in record])

# records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(records_np)]
records_dict=read_data

# _len = len(records_np)
db.insert_records(records_dict)
db_ids=db.retrive(query, top_k=1)
db_ids_minus_one = [id - 1 for id in db_ids]
res = run_queries(db_ids_minus_one, records_np, 1, 1, np.array([query]))
print(eval(res))
